🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys. **You will need to ensure you've executed the Indexing pipeline before completing this exercise**

# Systematic RAG Evaluation: Naive vs Hybrid Comparison

## 📋 Overview

This notebook demonstrates a **systematic evaluation workflow** for comparing two RAG (Retrieval-Augmented Generation) approaches using **Haystack custom components**. We'll create a reproducible pipeline to:

1. **Load evaluation datasets** from CSV files
2. **Process queries** through both Naive and Hybrid RAG SuperComponents 
3. **Generate comprehensive metrics** using the RAGAS framework
4. **Compare performance** systematically

## 🎯 Learning Objectives

By the end of this notebook, you will understand how to:
- Create reusable evaluation components for RAG systems
- Build scalable pipelines for systematic RAG comparison
- Interpret RAGAS metrics in comparative context
- Make data-driven decisions between RAG approaches

## 🔧 Evaluation Pipeline

Our pipeline consists of three main components:

```
CSV Data → RAGDataAugmenter → RagasEvaluation → Metrics & Results
```

**Key Benefits:**
- **Systematic**: Same evaluation conditions for both RAG systems
- **Reproducible**: Consistent evaluation across experiments
- **Scalable**: Easy to add new RAG implementations
- **Comprehensive**: Multiple metrics provide complete assessment

---

## Component 1: CSV Data Loader 📊

The **CSVReaderComponent** serves as the entry point for our evaluation pipeline. It handles loading synthetic evaluation datasets and ensures data quality before processing.

**Key Features:**
- **Robust Error Handling**: Validates file existence and data integrity
- **Pandas Integration**: Returns data as DataFrame for easy manipulation
- **Pipeline Compatible**: Designed to work seamlessly with Haystack pipelines

**Input:** File path to CSV containing evaluation queries and ground truth
**Output:** Pandas DataFrame ready for RAG processing

In [1]:
import pandas as pd
from pathlib import Path
from haystack import component, Pipeline
from typing import List, Optional, Dict, Any, Union

@component
class CSVReaderComponent:
    """Reads a CSV file into a Pandas DataFrame."""

    @component.output_types(data_frame=pd.DataFrame)
    def run(self, source: Union[str, Path]):
        """
        Reads the CSV file from the first source in the list.
        
        Args:
            sources: List of file paths to CSV files. Only the first file will be processed.
            
        Returns:
            dict: Dictionary containing the loaded DataFrame under 'data_frame' key.
            
        Raises:
            FileNotFoundError: If the file doesn't exist or can't be read.
            ValueError: If the DataFrame is empty after loading.
        """
        if not source:
            raise ValueError("No sources provided")
            

        try:
            df = pd.read_csv(source)
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found at {source}")
        except Exception as e:
            raise ValueError(f"Error reading CSV file {source}: {str(e)}")

        # Check if DataFrame is empty using proper pandas method
        if df.empty:
            raise ValueError(f"DataFrame is empty after loading from {source}")

        print(f"Loaded DataFrame with {len(df)} rows from {source}.")
        return {"data_frame": df}

## Component 2: RAG Data Augmentation 🔄

The **RAGDataAugmenterComponent** is the core of our evaluation workflow. It takes each query from our evaluation dataset and processes it through a RAG SuperComponent, collecting both the generated responses and retrieved contexts.

**🔑 Key Design Decisions:**

1. **SuperComponent Flexibility**: Accepts any pre-configured RAG SuperComponent (Naive, Hybrid, or custom)
2. **Batch Processing**: Efficiently processes entire evaluation datasets
3. **Data Augmentation**: Enriches the original dataset with RAG outputs for evaluation
4. **Context Extraction**: Captures retrieved documents for context-based metrics

**Pipeline Integration:**
- **Input**: DataFrame with queries from CSVReaderComponent  
- **Process**: Runs each query through the specified RAG SuperComponent
- **Output**: Augmented DataFrame with responses and retrieved contexts

**💡 Why This Approach?**
By separating RAG execution from evaluation, we can:
- **Swap RAG systems** without changing evaluation logic
- **Cache RAG results** for multiple evaluation runs  
- **Debug RAG performance** independently of metrics calculation
- **Scale evaluation** across different datasets and configurations

In [3]:
from haystack import SuperComponent, super_component

@component
class RAGDataAugmenterComponent:
    """
    Applies a RAG SuperComponent to each query in a DataFrame and 
    augments the data with the generated answer and retrieved contexts.
    """

    def __init__(self, rag_supercomponent: SuperComponent):
        # We store the pre-initialized SuperComponent
        self.rag_supercomponent = rag_supercomponent
        self.output_names = ["augmented_data_frame"]

    @component.output_types(augmented_data_frame=pd.DataFrame)
    def run(self, data_frame: pd.DataFrame):
        
        # New columns to store RAG results
        answers: List[str] = []
        contexts: List[List[str]] = []

        print(f"Running RAG SuperComponent on {len(data_frame)} queries...")

        # Iterate through the queries (user_input column)
        for _, row in data_frame.iterrows():
            query = row["user_input"]
            
            # 1. Run the RAG SuperComponent
            # It expects 'query' as input and returns a dictionary.
            rag_output = self.rag_supercomponent.run(query=query)
            
            # 2. Extract answer and contexts
            # Based on the naive_rag_sc/hybrid_rag_sc structure:
            answer = rag_output.get('replies', [''])[0]
            
            # Extract content from the Document objects
            retrieved_docs = rag_output.get('documents', [])
            retrieved_contexts = [doc.content for doc in retrieved_docs]
            
            answers.append(answer)
            contexts.append(retrieved_contexts)
        
        # 3. Augment the DataFrame
        data_frame['response'] = answers
        data_frame['retrieved_contexts'] = contexts
        
        print("RAG processing complete.")
        return {"augmented_data_frame": data_frame}

## Component 3: RAGAS Evaluation Engine 📈

The **RagasEvaluationComponent** integrates the RAGAS framework into our Haystack pipeline, providing systematic evaluation metrics for RAG systems.

**🎯 Core Evaluation Metrics:**

| Metric | Purpose | What It Measures |
|--------|---------|------------------|
| **Faithfulness** | Response Quality | Factual consistency with retrieved context |
| **ResponseRelevancy** | Relevance | How well responses answer the questions |
| **LLMContextRecall** | Retrieval Quality | How well retrieval captures relevant information |
| **FactualCorrectness** | Accuracy | Correctness of factual claims in responses |

**🔧 Technical Implementation:**
- **Focused Metrics**: Core metrics for reliable comparison
- **LLM Integration**: Uses OpenAI GPT models for evaluation judgments  
- **Data Format Handling**: Automatically formats data for RAGAS requirements
- **Comprehensive Output**: Returns both aggregated metrics and detailed per-query results

In [4]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy

from ragas.llms import llm_factory
from haystack.utils import Secret
import os
from ragas.llms import HaystackLLMWrapper
from haystack.components.generators import OpenAIGenerator

@component
class RagasEvaluationComponent:
    """
    Prepares data for Ragas, runs the evaluation, and returns the metrics.
    Simplified for core metrics comparison.
    """
    
    def __init__(self, 
                 metrics: Optional[List[Any]] = None,
                 llm_model: str = "gpt-4o-mini",
                 openai_api_key: Optional[str] = None):
        """
        Initialize the RagasEvaluationComponent.
        
        Args:
            metrics: List of RAGAS metrics to evaluate (defaults to core metrics)
            llm_model (str): OpenAI model for evaluation. Defaults to "gpt-4o-mini".
            openai_api_key (Optional[str]): OpenAI API key. If None, will use environment variable.
        """
        
        # Default to core metrics for systematic comparison
        if metrics is None:
            self.metrics = [
                Faithfulness(), 
                ResponseRelevancy(),
                LLMContextRecall(),
                FactualCorrectness()
            ]
        else:
            self.metrics = metrics
            
        self.llm_model = llm_model
        self.openai_api_key = openai_api_key or os.getenv('OPENAI_API_KEY')
        
        if not self.openai_api_key:
            raise ValueError("OpenAI API key not found. Please set OPENAI_API_KEY environment variable or pass openai_api_key parameter.")
        
        # Configure RAGAS LLM for evaluation
        self.ragas_llm = HaystackLLMWrapper(
            OpenAIGenerator(
                model=self.llm_model,
                api_key=Secret.from_token(self.openai_api_key)
            )
        )

    @component.output_types(metrics=Dict[str, float], evaluation_df=pd.DataFrame)
    def run(self, augmented_data_frame: pd.DataFrame):
        
        # 1. Map columns to Ragas requirements
        ragas_data = pd.DataFrame({
            'user_input': augmented_data_frame['user_input'],
            'response': augmented_data_frame['response'], 
            'retrieved_contexts': augmented_data_frame['retrieved_contexts'],
            'reference': augmented_data_frame['reference'],
            'reference_contexts': augmented_data_frame['reference_contexts'].apply(eval)
        })

        print("Creating Ragas EvaluationDataset...")
        # 2. Create EvaluationDataset
        dataset = EvaluationDataset.from_pandas(ragas_data)

        print("Starting Ragas evaluation...")
        
        # 3. Run Ragas Evaluation
        results = evaluate(
            dataset=dataset,
            metrics=self.metrics,
            llm=self.ragas_llm
        )
        
        results_df = results.to_pandas()
        
        print("Ragas evaluation complete.")
        print(f"Overall metrics: {results}")
        
        return {"metrics": results, "evaluation_df": results_df}

---

# 🧪 Systematic RAG Comparison: Naive vs Hybrid

Now we'll systematically evaluate both RAG SuperComponents using the same evaluation pipeline. This ensures fair comparison with identical evaluation conditions.

## 🎯 Comparison Strategy

Our approach enables **systematic comparison**:

1. **Same Dataset**: Both systems evaluated on identical test queries
2. **Same Metrics**: Consistent evaluation criteria across both approaches
3. **Same Pipeline**: Identical processing workflow eliminates bias
4. **Reproducible Results**: Pipeline ensures consistent evaluation conditions

## 📊 Dataset Information

We'll use a synthetic evaluation dataset:
- **`synthetic_tests_advanced_branching_2.csv`**: Focused dataset for comparison

**Dataset Structure:**
- `user_input`: Questions to ask the RAG system
- `reference`: Ground truth answers for comparison
- `reference_contexts`: Expected retrieved contexts

### Setup: Initialize Both RAG SuperComponents

First, we'll initialize both RAG SuperComponents with consistent parameters for fair comparison.

**🔧 Configuration:**
- **Same base parameters**: Both systems use identical core settings
- **Document store**: Shared Elasticsearch document store
- **Models**: Consistent LLM and embedding models for both systems

In [5]:
# --- Setup Environment & Dependencies ---
from scripts.rag.hybridrag import HybridRAGSuperComponent
from scripts.rag.naiverag import NaiveRAGSuperComponent
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
import os

# Initialize document store
document_store = ElasticsearchDocumentStore(hosts="http://localhost:9200")

# Create both RAG SuperComponents with base parameters for fair comparison
naive_rag_sc = NaiveRAGSuperComponent(
    document_store=document_store
)

hybrid_rag_sc = HybridRAGSuperComponent(
    document_store=document_store
)

print("✅ Both RAG SuperComponents initialized successfully!")
print(f"📊 Naive RAG: {naive_rag_sc.__class__.__name__}")
print(f"📊 Hybrid RAG: {hybrid_rag_sc.__class__.__name__}")

✅ Both RAG SuperComponents initialized successfully!
📊 Naive RAG: NaiveRAGSuperComponent
📊 Hybrid RAG: HybridRAGSuperComponent


In [6]:
# --- Create RAG Evaluation SuperComponent for Systematic Comparison ---

@super_component
class RAGEvaluationSuperComponent:
    def __init__(self, 
                 rag_supercomponent, 
                 system_name: str,
                 llm_model: str = "gpt-4o-mini",
                 openai_api_key: Optional[str] = None):
        """
        Initialize the RAG Evaluation SuperComponent.
        
        Args:
            rag_supercomponent: The RAG system to evaluate
            system_name (str): Name for logging and identification
            llm_model (str): OpenAI model for evaluation. Defaults to "gpt-4o-mini".
            openai_api_key (Optional[str]): OpenAI API key. If None, will use environment variable.
        """
        self.rag_supercomponent = rag_supercomponent
        self.system_name = system_name
        self.llm_model = llm_model
        self.openai_api_key = openai_api_key or os.getenv('OPENAI_API_KEY')
        
        if not self.openai_api_key:
            raise ValueError("OpenAI API key not found. Please set OPENAI_API_KEY environment variable or pass openai_api_key parameter.")
        
        self._build_pipeline()
    
    def _build_pipeline(self):
        """Build the RAG evaluation pipeline with initialized components."""
        
        print(f"\n🔄 Building evaluation pipeline for {self.system_name}...")
        print("=" * 50)
        
        # --- 1. Initialize Evaluation Pipeline Components ---
        
        # CSV Reader: Loads evaluation dataset
        reader = CSVReaderComponent()
        
        # RAG Data Augmenter: Processes queries through the RAG system
        augmenter = RAGDataAugmenterComponent(rag_supercomponent=self.rag_supercomponent)
        
        # RAGAS Evaluator: Computes evaluation metrics
        evaluator = RagasEvaluationComponent(
            llm_model=self.llm_model,
            openai_api_key=self.openai_api_key
        )
        
        # --- 2. Build the Evaluation Pipeline ---
        self.pipeline = Pipeline()
        
        # Add all components to the pipeline
        self.pipeline.add_component("reader", reader)
        self.pipeline.add_component("augmenter", augmenter)
        self.pipeline.add_component("evaluator", evaluator)
        
        # --- 3. Connect the Components in a Graph ---
        
        # CSV Data -> RAG Augmentation -> RAGAS Evaluation
        self.pipeline.connect("reader.data_frame", "augmenter.data_frame")
        self.pipeline.connect("augmenter.augmented_data_frame", "evaluator.augmented_data_frame")
        
        # --- 4. Define Input and Output Mappings ---
        self.input_mapping = {
            "csv_source": ["reader.source"]
        }

        self.output_mapping = {
            "evaluator.metrics": "metrics",
            "evaluator.evaluation_df": "evaluation_df"
        }
        
        print(f"✅ Evaluation pipeline for {self.system_name} built successfully!")



## Experiment 1: Naive RAG Evaluation 🔬

Let's evaluate the Naive RAG SuperComponent first. This will establish our baseline performance metrics.

**🎯 What to Observe:**
- Processing time and efficiency
- Core metric scores (Faithfulness, Relevancy, Context Recall, Factual Correctness)
- Any errors or warnings during evaluation

In [7]:
# Evaluate Naive RAG SuperComponent
csv_file_path = "data_for_eval/synthetic_tests_advanced_branching_10.csv"

# Create evaluation SuperComponent (csv_source now passed in run method)
evaluation_sc = RAGEvaluationSuperComponent(
    rag_supercomponent=naive_rag_sc,
    system_name="Naive RAG"
)

# Run evaluation with CSV source passed to run method
naive_results = evaluation_sc.run(csv_source=csv_file_path)

print(f"✅ Naive RAG evaluation complete!")


🔄 Building evaluation pipeline for Naive RAG...
✅ Evaluation pipeline for Naive RAG built successfully!
Loaded DataFrame with 10 rows from data_for_eval/synthetic_tests_advanced_branching_10.csv.
Running RAG SuperComponent on 10 queries...
RAG processing complete.
Creating Ragas EvaluationDataset...
Starting Ragas evaluation...
RAG processing complete.
Creating Ragas EvaluationDataset...
Starting Ragas evaluation...


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 g

Ragas evaluation complete.
Overall metrics: {'faithfulness': 0.6411, 'answer_relevancy': 0.6678, 'context_recall': 0.6800, 'factual_correctness(mode=f1)': 0.3556}
✅ Naive RAG evaluation complete!


### Naive RAG Results Analysis 📊

Let's examine the detailed evaluation results from our Naive RAG system.

In [8]:
# Display Naive RAG detailed results
print("📊 Naive RAG - Detailed Results:")
naive_results['evaluation_df']

📊 Naive RAG - Detailed Results:


,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_recall,factual_correctness(mode=f1)
0,Wut is Meta AI and why do peeple have concerns...,"[What is AI, how does it work and why are some...","[What is AI, how does it work and why are some...",Meta AI refers to the artificial intelligence ...,Meta AI is a generative AI tool that can hold ...,0.928571,0.924937,1.0,0.60
1,What are the implications of AI technology on ...,"[In the same vein, the IEEE Global Initiative ...",[Why is AI controversial?\nWhile acknowledging...,I don't have enough information to answer.,The controversy surrounding AI technology incl...,0.000000,0.000000,0.0,0.00
2,What does the Artificial Intelligence Act regu...,[AI will reconfigure how society and the econo...,[Are there laws governing AI?\nSome government...,The Artificial Intelligence Act regulates high...,The EU's Artificial Intelligence Act places co...,0.500000,1.000000,1.0,0.80
3,What steps are federal officials considering t...,[A body of case law has shown that the situati...,[<1-hop>\n\nCreate a federal AI advisory commi...,Federal officials are considering several step...,Federal officials are considering the creation...,1.000000,0.977361,1.0,0.40
4,What are the trends in user satisfaction regar...,[The yellow line represents the first cohort o...,[<1-hop>\n\n5.5 Quality of Interactions\nWe ad...,I don't have enough information to answer rega...,User satisfaction regarding Technical Help int...,1.000000,0.000000,0.0,0.00
5,How does the quality of interactions in Techni...,[Figure 9 disaggregates four of the seven Conv...,[<1-hop>\n\n5.5 Quality of Interactions\nWe ad...,I don't have enough information to answer.,The quality of interactions in Technical Help ...,0.000000,0.000000,0.0,0.00
6,What are the ethical implications and environm...,"[What is AI, how does it work and why are some...","[<1-hop>\n\nWhat is AI, how does it work and w...",The rapid growth of artificial intelligence (A...,The rapid growth of artificial intelligence (A...,NaN,0.981306,1.0,NaN
7,How do the user message classifications relate...,"[Teams, Enterprise, Education), which we do no...",[<1-hop>\n\nThe yellow line represents the fir...,The user message classifications highlight a s...,The user message classifications indicate that...,1.000000,0.911098,0.8,0.58
8,What trends can be observed in the user messag...,"[Teams, Enterprise, Education), which we do no...",[<1-hop>\n\nThe yellow line represents the fir...,The trends observed in the user message classi...,The trends observed in the user message classi...,0.700000,0.945481,1.0,0.50
9,In what ways do the challenges of data access ...,[Smart cities\nMetropolitan governments are us...,[<1-hop>\n\nSmart cities\nMetropolitan governm...,The challenges of data access problems signifi...,The challenges of data access problems signifi...,NaN,0.937816,1.0,0.32


In [9]:
# Display Naive RAG summary metrics
print("📈 Naive RAG - Summary Metrics:")
naive_metrics = naive_results['metrics']
naive_metrics

📈 Naive RAG - Summary Metrics:


{'faithfulness': 0.6411, 'answer_relevancy': 0.6678, 'context_recall': 0.6800, 'factual_correctness(mode=f1)': 0.3556}

## Experiment 2: Hybrid RAG Evaluation 🔬⚡

Now let's evaluate the **Hybrid RAG SuperComponent** using the identical evaluation pipeline. This systematic approach ensures fair comparison.

**🔄 Key Benefits of This Approach:**
- **Identical Conditions**: Same pipeline, metrics, and dataset
- **Systematic Comparison**: Eliminates evaluation bias
- **Reproducible Results**: Consistent methodology across both systems

**🎯 Expected Improvements:**
Hybrid RAG typically shows better performance due to:
- **Dense + Sparse Retrieval**: Combines semantic and keyword-based search
- **Enhanced Context Quality**: Better retrieval often leads to better responses
- **Improved Robustness**: Multiple retrieval methods reduce failure modes

# Evaluate Hybrid RAG SuperComponent


In [10]:
# Evaluate Hybrid RAG SuperComponent

# Create evaluation SuperComponent (csv_source now passed in run method)
hybrid_evaluation_sc = RAGEvaluationSuperComponent(
    rag_supercomponent=hybrid_rag_sc,
    system_name="Hybrid RAG"
)

# Run evaluation with CSV source passed to run method
hybrid_results = hybrid_evaluation_sc.run(csv_source=csv_file_path)

print(f"✅ Hybrid RAG evaluation complete!")


🔄 Building evaluation pipeline for Hybrid RAG...
✅ Evaluation pipeline for Hybrid RAG built successfully!
Loaded DataFrame with 10 rows from data_for_eval/synthetic_tests_advanced_branching_10.csv.
Running RAG SuperComponent on 10 queries...
RAG processing complete.
Creating Ragas EvaluationDataset...
Starting Ragas evaluation...


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Ragas evaluation complete.
Overall metrics: {'faithfulness': 0.9626, 'answer_relevancy': 0.7374, 'context_recall': 0.7633, 'factual_correctness(mode=f1)': 0.4090}
✅ Hybrid RAG evaluation complete!


In [13]:
# Display Hybrid RAG detailed results
print("📊 Hybrid RAG - Detailed Results:")
hybrid_results['evaluation_df']

📊 Hybrid RAG - Detailed Results:


,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_recall,factual_correctness(mode=f1)
0,Wut is Meta AI and why do peeple have concerns...,"[What is AI, how does it work and why are some...","[What is AI, how does it work and why are some...",Meta AI is a generative AI tool developed by M...,Meta AI is a generative AI tool that can hold ...,0.928571,0.889708,1.000000,0.48
1,What are the implications of AI technology on ...,"[What is AI, how does it work and why are some...",[Why is AI controversial?\nWhile acknowledging...,I don't have enough information to answer.,The controversy surrounding AI technology incl...,1.000000,0.000000,0.500000,0.00
2,What does the Artificial Intelligence Act regu...,"[- James Kurose, “Testimony before the House C...",[Are there laws governing AI?\nSome government...,"The ""Future of Artificial Intelligence Act"" ai...",The EU's Artificial Intelligence Act places co...,1.000000,0.908452,0.500000,0.00
3,What steps are federal officials considering t...,[AI will reconfigure how society and the econo...,[<1-hop>\n\nCreate a federal AI advisory commi...,Federal officials are considering the establis...,Federal officials are considering the creation...,0.928571,0.912998,0.500000,0.50
4,What are the trends in user satisfaction regar...,[Figure 9 disaggregates four of the seven Conv...,[<1-hop>\n\n5.5 Quality of Interactions\nWe ad...,The trends in user satisfaction regarding Tech...,User satisfaction regarding Technical Help int...,1.000000,0.944705,0.666667,0.62
5,How does the quality of interactions in Techni...,"[Overall, the majority of ChatGPT usage\nat wo...",[<1-hop>\n\n5.5 Quality of Interactions\nWe ad...,I don't have enough information to answer.,The quality of interactions in Technical Help ...,1.000000,0.000000,0.666667,0.00
6,What are the ethical implications and environm...,"[What is AI, how does it work and why are some...","[<1-hop>\n\nWhat is AI, how does it work and w...",Experts have raised several ethical implicatio...,The rapid growth of artificial intelligence (A...,1.000000,0.979982,1.000000,0.62
7,How do the user message classifications relate...,"[Teams, Enterprise, Education), which we do no...",[<1-hop>\n\nThe yellow line represents the fir...,The user message classifications reveal signif...,The user message classifications indicate that...,1.000000,0.875714,0.800000,0.52
8,What trends can be observed in the user messag...,"[Teams, Enterprise, Education), which we do no...",[<1-hop>\n\nThe yellow line represents the fir...,The trends observed in the user message classi...,The trends observed in the user message classi...,0.769231,0.924762,1.000000,0.89
9,In what ways do the challenges of data access ...,[AI will reconfigure how society and the econo...,[<1-hop>\n\nSmart cities\nMetropolitan governm...,The challenges of data access problems signifi...,The challenges of data access problems signifi...,1.000000,0.937708,1.000000,0.46


In [14]:
# Display Hybrid RAG summary metrics
print("📈 Hybrid RAG - Summary Metrics:")
hybrid_metrics = hybrid_results['metrics']
hybrid_metrics

📈 Hybrid RAG - Summary Metrics:


{'faithfulness': 0.9626, 'answer_relevancy': 0.7374, 'context_recall': 0.7633, 'factual_correctness(mode=f1)': 0.4090}

## 📊 Side-by-Side Performance Comparison

Let's create a comprehensive comparison of both RAG systems to understand their relative performance across all metrics.

In [15]:
# --- Create Side-by-Side Performance Comparison ---
import pandas as pd
import numpy as np

# Extract average metrics from both evaluations using proper RAGAS data structure methods
# The EvaluationResult object contains individual scores for each query
naive_scores = naive_metrics.scores
hybrid_scores = hybrid_metrics.scores

print("🔍 Processing individual query scores...")
print(f"Naive RAG: {len(naive_scores)} queries evaluated")
print(f"Hybrid RAG: {len(hybrid_scores)} queries evaluated")

# Compute averages for each metric using pandas for easier calculation
naive_df = pd.DataFrame(naive_scores)
hybrid_df = pd.DataFrame(hybrid_scores)

# Calculate mean values for each metric
naive_averages = naive_df.mean()
hybrid_averages = hybrid_df.mean()

print("\n📊 Average Scores Computed:")
print(f"Naive RAG averages: {dict(naive_averages)}")
print(f"Hybrid RAG averages: {dict(hybrid_averages)}")

# Create comparison DataFrame
comparison_data = {
    'Metric': list(naive_averages.index),
    'Naive RAG': list(naive_averages.values),
    'Hybrid RAG': list(hybrid_averages.values)
}

# Calculate improvement percentages
comparison_df = pd.DataFrame(comparison_data)
comparison_df['Improvement (%)'] = ((comparison_df['Hybrid RAG'] - comparison_df['Naive RAG']) / comparison_df['Naive RAG'] * 100).round(2)
comparison_df['Better System'] = comparison_df.apply(
    lambda row: '🏆 Hybrid RAG' if row['Hybrid RAG'] > row['Naive RAG'] 
    else '🏆 Naive RAG' if row['Naive RAG'] > row['Hybrid RAG'] 
    else '🤝 Tie', axis=1
)

# Round scores for better display
comparison_df['Naive RAG'] = comparison_df['Naive RAG'].round(4)
comparison_df['Hybrid RAG'] = comparison_df['Hybrid RAG'].round(4)

print("\n" + "=" * 80)
print("🔍 COMPREHENSIVE RAG SYSTEM COMPARISON")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("\n" + "=" * 80)

# Calculate overall winner
hybrid_wins = sum(comparison_df['Hybrid RAG'] > comparison_df['Naive RAG'])
naive_wins = sum(comparison_df['Naive RAG'] > comparison_df['Hybrid RAG'])
ties = sum(comparison_df['Naive RAG'] == comparison_df['Hybrid RAG'])

print(f"\n🏁 FINAL SCORECARD:")
print(f"🏆 Hybrid RAG wins: {hybrid_wins} metrics")
print(f"🏆 Naive RAG wins: {naive_wins} metrics") 
print(f"🤝 Ties: {ties} metrics")

if hybrid_wins > naive_wins:
    print(f"\n🎉 OVERALL WINNER: Hybrid RAG SuperComponent!")
    print(f"   Better performance in {hybrid_wins}/{len(comparison_df)} metrics")
elif naive_wins > hybrid_wins:
    print(f"\n🎉 OVERALL WINNER: Naive RAG SuperComponent!")  
    print(f"   Better performance in {naive_wins}/{len(comparison_df)} metrics")
else:
    print(f"\n🤝 RESULT: It's a tie between both systems!")
    
avg_improvement = comparison_df['Improvement (%)'].mean()
print(f"\n📈 Average improvement by Hybrid RAG: {avg_improvement:.2f}%")

# Show standard deviations to understand score consistency
naive_stds = naive_df.std()
hybrid_stds = hybrid_df.std()

print(f"\n📊 Score Consistency (Standard Deviation):")
for metric in naive_averages.index:
    print(f"   {metric}:")
    print(f"     Naive RAG: {naive_stds[metric]:.4f}")
    print(f"     Hybrid RAG: {hybrid_stds[metric]:.4f}")
    stability_winner = "Hybrid RAG" if hybrid_stds[metric] < naive_stds[metric] else "Naive RAG"
    print(f"     More consistent: {stability_winner}")

🔍 Processing individual query scores...
Naive RAG: 10 queries evaluated
Hybrid RAG: 10 queries evaluated

📊 Average Scores Computed:
Naive RAG averages: {'faithfulness': np.float64(0.6410714285714286), 'answer_relevancy': np.float64(0.6678000145616547), 'context_recall': np.float64(0.6799999999999999), 'factual_correctness(mode=f1)': np.float64(0.35555555555555557)}
Hybrid RAG averages: {'faithfulness': np.float64(0.9626373626373628), 'answer_relevancy': np.float64(0.7374028997143484), 'context_recall': np.float64(0.7633333333333333), 'factual_correctness(mode=f1)': np.float64(0.4090000000000001)}

🔍 COMPREHENSIVE RAG SYSTEM COMPARISON
                      Metric  Naive RAG  Hybrid RAG  Improvement (%) Better System
                faithfulness     0.6411      0.9626            50.16  🏆 Hybrid RAG
            answer_relevancy     0.6678      0.7374            10.42  🏆 Hybrid RAG
              context_recall     0.6800      0.7633            12.25  🏆 Hybrid RAG
factual_correctness(mode